In [13]:
import numpy as np
import cPickle
import math
import sys
import emcee
import scipy.optimize as op
import matplotlib.pyplot as plt
import corner
from scipy import integrate
%matplotlib inline

In [15]:
def pickle_to_file(data, fname):
    """Save a variable simply to a file"""
    try:
        fh = open(fname, 'w')
        cPickle.dump(data, fh)
        fh.close()
    except:
        print "Pickling failed!", sys.exc_info()[0]

def pickle_from_file(fname):
    """Restore a variable saved with pickle_to_file"""
    try:
        fh = open(fname, 'r')
        data = cPickle.load(fh)
        fh.close()
    except:
        print "Loading pickled data failed!", sys.exc_info()[0]
        data = None

    return data


def lnL(theta, x, y, yerr):
    model = np.polynomial.polynomial.polyval(x, theta)
    
    #print "model is= {0}".format(model)
    
    inv_sigma2 = 1.0/(yerr**2)
    return -0.5*(np.sum((y-model)**2*inv_sigma2))

def lnprior(theta):
    for elements in theta:
        if -100 < elements < 100:
            #return np.log(limits_for_prior**-len(theta)) #PREGUNTAR !!!!!!!!
            return np.log(200)
    return -np.inf

def lnprob(theta, x, y, yerr):
    lp = lnprior(theta)    
    if not np.isfinite(lp):
        return -np.inf
    return lp+lnL(theta, x, y, yerr)

def integrate_posterior_2D(lnprop, theta0, theta1, xlim, ylim, x, y, yerr):
    func = lambda theta0, theta1: np.exp(lnprob([theta0, theta1], x, y, yerr))
    return integrate.dblquad(func, xlim[0], xlim[1],
                             lambda x: ylim[0], lambda x: ylim[1])

In [9]:
#Dictionary
d = pickle_from_file('data-for-poly-test.pkl')
#print d

In [32]:
theta0, theta1 = [-0.3270517196132437, 1.5441764461723155]
xlim=[-0.47086515425428987, -0.15493311046325484]
ylim=[1.3403336188690207, 1.7588797074872327]

func = lambda theta0, theta1: np.exp(lnprob([theta0, theta1], d['x'], d['y'], d['sigma_y']))
integrate.dblquad(func,-1, 1,
                             lambda x: 0, lambda x: 1)

(1.2563242699816897e-26, 2.5970459055507145e-24)